In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# ডাটা তৈরি
X, y = make_classification(n_samples=1000, n_features=4, n_classes=2, random_state=1)

# ইনপুট ফিচার সংখ্যা নির্ধারণ
n_features = X.shape[1]

# ডাটাকে ট্রেনিং এবং টেস্টে ভাগ করা
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ডাটা স্কেল করা (StandardScaler)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# PyTorch টেনসর তৈরি
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# মডেল তৈরি (PyTorch)
class BinaryClassifier(nn.Module):
    def __init__(self, input_size):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 10)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(10, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# মডেল ইনস্ট্যান্স তৈরি
model = BinaryClassifier(n_features)

# লস ফাংশন এবং অপটিমাইজার সেট করা
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.SGD(model.parameters(), lr=0.01)

# মডেল প্রশিক্ষণ
epochs = 50
for epoch in range(epochs):
    model.train()
    
    # ফোরওয়ার্ড পাস
    outputs = model(X_train_tensor)
    
    # লস ক্যালকুলেশন
    loss = criterion(outputs, y_train_tensor)
    
    # গ্র্যাডিয়েন্ট শূন্য করা
    optimizer.zero_grad()
    
    # ব্যাকওয়ার্ড পাস
    loss.backward()
    
    # আপডেট করা
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# মডেল সংরক্ষণ
torch.save(model.state_dict(), 'model.pth')

# মডেল লোড করা
loaded_model = BinaryClassifier(n_features)
loaded_model.load_state_dict(torch.load('model.pth'))
loaded_model.eval()

# নতুন ইনপুটের জন্য প্রেডিকশন
new_data = np.array([[1.91518414, 1.14995454, -1.52847073, 0.79430654]])
new_data = scaler.transform(new_data)  # স্কেলিং করা
new_data_tensor = torch.tensor(new_data, dtype=torch.float32)

# প্রেডিকশন
with torch.no_grad():
    prediction = loaded_model(new_data_tensor)
    print("Prediction:", prediction.item())


Epoch [10/50], Loss: 0.7249
Epoch [20/50], Loss: 0.7204
Epoch [30/50], Loss: 0.7160
Epoch [40/50], Loss: 0.7118
Epoch [50/50], Loss: 0.7076
Prediction: 0.45129936933517456


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from pandas import read_csv
import numpy as np
from sklearn.preprocessing import StandardScaler

# ডাটা লোড করা
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv'
df = read_csv(path, header=None)

# ইনপুট এবং আউটপুট ভাগ করা
X, y = df.values[:, :-1], df.values[:, -1]

# ডাটাকে ফ্লোট টাইপে রূপান্তর করা
X = X.astype('float32')

# টেক্সট লেবেলকে নাম্বারে রূপান্তর করা
y = LabelEncoder().fit_transform(y)

# ডাটাকে ট্রেনিং ও টেস্ট সেটে ভাগ করা
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

# ইনপুট ফিচার সংখ্যা নির্ধারণ
n_features = X_train.shape[1]

# ডাটা স্কেল করা (StandardScaler)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# PyTorch টেনসর তৈরি
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)  # Categorical labels
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# মডেল তৈরি (PyTorch)
class MultiClassClassifier(nn.Module):
    def __init__(self, input_size, output_size):
        super(MultiClassClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 10)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(10, output_size)  # Output layer for multi-class
        self.softmax = nn.Softmax(dim=1)  # Softmax for multi-class classification

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

# মডেল ইনস্ট্যান্স তৈরি
model = MultiClassClassifier(n_features, 3)  # 3 classes for Iris dataset

# লস ফাংশন এবং অপটিমাইজার সেট করা
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.01)

# মডেল প্রশিক্ষণ
epochs = 50
for epoch in range(epochs):
    model.train()

    # ফোরওয়ার্ড পাস
    outputs = model(X_train_tensor)

    # লস ক্যালকুলেশন
    loss = criterion(outputs, y_train_tensor)

    # গ্র্যাডিয়েন্ট শূন্য করা
    optimizer.zero_grad()

    # ব্যাকওয়ার্ড পাস
    loss.backward()

    # আপডেট করা
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# মডেল মূল্যায়ন
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)  # Get the class with highest probability
    correct = (predicted == y_test_tensor).sum().item()
    accuracy = correct / y_test_tensor.size(0)

print("Test Accuracy:", accuracy)


Epoch [10/50], Loss: 1.0239
Epoch [20/50], Loss: 0.8940
Epoch [30/50], Loss: 0.8026
Epoch [40/50], Loss: 0.7551
Epoch [50/50], Loss: 0.7267
Test Accuracy: 0.86
